In [2]:
import os
import pandas as pd

DATA_PATH = "G:\내 드라이브\Project\RGBLab\ABB\Data"

import xml.etree.ElementTree as ET

def parse_xml_for_actionname(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    action_names = []

    for action in root.findall(".//action"):
        action_name_element = action.find("actionname")
        if action_name_element is not None:
            action_names.append(action_name_element.text)

    return action_names


df = pd.read_csv(os.path.join("./processed_data", 'data_files.csv'))
actions = set()
for index, row in df.iterrows():
    xml_file = os.path.join(DATA_PATH, row['path'], row['filename']+'.xml')
    try:
        action_names = parse_xml_for_actionname(xml_file)
        actions.update(action_names)
    except FileNotFoundError:
        print(f"File {xml_file} not found.")
    except ET.ParseError:
        print(f"Error parsing {xml_file}")

print("Unique action names:", actions)

Unique action names: {'piercing', 'stop and go', 'punching', 'threaten', 'kicking', 'running', 'pulling', 'around', 'climbwall', 'pushing', 'throwing', 'walking', 'falldown'}


In [1]:
import torch
%load_ext autoreload
%autoreload 2

In [4]:
from combined_flow_extractor import CombinedFlowModel
model = CombinedFlowModel().to(device='cuda:3')

ImportError: /home/ict06/.conda/envs/torch21/lib/python3.8/site-packages/spatial_correlation_sampler_backend.cpython-38-x86_64-linux-gnu.so: undefined symbol: _ZNK2at6Tensor6deviceEv

In [1]:
import torch
%cd ..

/home/ict06/dev/abb_project


In [2]:
from bn_inception import get_bninception
from combined_flow_extractor import CombinedFlowModel
from resnet import resnet50

In [3]:
class Extractor(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = resnet50()
        self.flow = CombinedFlowModel()
    
    def forward(self, x):
        out_f = self.flow(x[:, :, : , :])
        out_r = self.resnet(x[:, :3, :, :])
        return torch.cat((out_f.squeeze(2,3), out_r), dim=1)

In [5]:
model = Extractor()
model.to('cuda:3')
model.eval()
import time
a = torch.randn(5, 6, 512, 512).to('cuda:3')
with torch.no_grad():
    st = time.time()
    for i in range(100):
        model(a)
    print((time.time()-st) * 10)

72.09044694900513


26.20076584815979


In [34]:
torch.cuda.memory_allocated('cuda:3') / 1024 / 1024

704.0556640625

In [35]:
from torch.profiler import profile, record_function, ProfilerActivity

In [39]:
with profile(activities=[ProfilerActivity.CUDA],
        profile_memory=True, record_shapes=True) as prof:
    model(a)

print(prof.key_averages().table(sort_by="self_cuda_memory_usage", row_limit=10))

STAGE:2023-10-28 01:07:01 3609271:3609271 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-10-28 01:07:01 3609271:3609271 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-10-28 01:07:01 3609271:3609271 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
[W collection.cpp:700] Warning: Failed to recover relationship between all profiler and kineto events: 1360 vs. 0  reassociated. (function reassociate)


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               [memory]         0.00%       0.000us         0.00%       0.000us       0.000us       0.000us         0.00%       0.000us       0.000us           0 b           0 b          1360  
                                  cudaStreamIsCapturing         0.15%      41.000us         0.1

In [40]:
print(prof.key_averages().table(sort_by="cuda_memory_usage", row_limit=10))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               [memory]         0.00%       0.000us         0.00%       0.000us       0.000us       0.000us         0.00%       0.000us       0.000us           0 b           0 b          1360  
                                  cudaStreamIsCapturing         0.15%      41.000us         0.1

In [41]:
import torch.utils.benchmark as benchmark

In [52]:
a = torch.randn(32, 6, 512, 512).to('cuda:3')
t0 = benchmark.Timer(
    stmt='with torch.no_grad(): model(a)',
    setup='from combined_flow_extractor import CombinedFlowModel; model=CombinedFlowModel();model.to("cuda:3")',
    globals={'a': a})


with torch.no_grad(): model(a)
setup: from combined_flow_extractor import CombinedFlowModel; model=CombinedFlowModel();model.to("cuda:3")
  238.76 ms
  1 measurement, 100 runs , 1 thread
